# Thread Configuration Performance Analysis

This notebook analyzes and visualizes performance data from different thread configurations for API client tests.

## Data Sources
- **reports/report#N.txt**: Summary statistics (overall duration, min/max/avg times)
- **reports/report#N.csv**: Detailed call-by-call measurements

## Visualizations
1. **Summary Comparison**: Bar chart of min/max/avg across all configurations
2. **Distribution Box Plot**: Shows variance and outliers in total call times
3. **Timeline Chart**: When tasks complete (reveals concurrency patterns)
4. **Breakdown Chart**: Average waiting time vs execution time
5. **Statistics Table**: Detailed metrics per configuration
6. **Side-by-Side Comparison**: Task-by-task comparison of specific configs

Run all cells to generate the full analysis.

In [46]:
%use lets-plot

import java.io.File

In [47]:
// Data classes for CSV parsing
data class CallData(
    val callId: String,
    val enqueuedAt: Long,
    val startedAt: Long,
    val completedAt: Long,
    val waitingTimeMs: Long,
    val execTimeMs: Long,
    val totalTimeMs: Long,
    val configId: Int,
    val configName: String
)

data class TestConfig(
    val id: Int,
    val name: String,
    val overallDuration: Long,
    val minCallTime: Long,
    val maxCallTime: Long,
    val avgCallTime: Long
)

In [48]:
// Load all reports
val reportsDir = File("reports")

fun loadTestConfig(reportNum: Int): TestConfig {
    val txtFile = File(reportsDir, "report#$reportNum.txt")
    val lines = txtFile.readLines()
    
    val name = lines[0].substringBefore("(")
    val overallDuration = lines[1].substringAfter(": ").toLong()
    val callExecTimes = lines[2].substringAfter(": ").split(", ")
    
    return TestConfig(
        id = reportNum,
        name = "#$reportNum: $name",  // Prepend report number to distinguish configs
        overallDuration = overallDuration,
        minCallTime = callExecTimes[0].toLong(),
        maxCallTime = callExecTimes[1].toLong(),
        avgCallTime = callExecTimes[2].toLong()
    )
}

fun loadCallData(reportNum: Int, configName: String): List<CallData> {
    val csvFile = File(reportsDir, "report#$reportNum.csv")
    return csvFile.readLines()
        .drop(1) // Skip header
        .map { line ->
            val parts = line.split(", ")
            CallData(
                callId = parts[0],
                enqueuedAt = parts[1].toLong(),
                startedAt = parts[2].toLong(),
                completedAt = parts[3].toLong(),
                waitingTimeMs = parts[4].toLong(),
                execTimeMs = parts[5].toLong(),
                totalTimeMs = parts[6].toLong(),
                configId = reportNum,
                configName = configName
            )
        }
}

// Load all data
val reportFiles = reportsDir.listFiles { _, name -> name.startsWith("report#") && name.endsWith(".txt") }
    ?.map { it.nameWithoutExtension.substringAfter("#").toInt() }
    ?.sorted() ?: emptyList()

val configs = reportFiles.map { loadTestConfig(it) }
val allCallData = reportFiles.flatMap { id -> 
    val config = configs.first { it.id == id }
    loadCallData(id, config.name)
}

println("Loaded ${configs.size} configurations with ${allCallData.size} total calls")
configs.forEach { println("  ${it.name}") }

Loaded 10 configurations with 200 total calls
  #0: Retrofit.singleThread
  #1: Retrofit.singleThread
  #2: Retrofit.singleThread
  #3: Retrofit.dedicatedThreads
  #4: Retrofit.dedicatedThreads
  #5: Retrofit.dedicatedThreads
  #6: Retrofit.dedicatedThreads
  #7: Retrofit.dedicatedThreads
  #8: Retrofit.dedicatedThreads
  #9: Retrofit.dedicatedThreads


In [49]:
// Summary comparison: min/max/avg call times
val summaryData = mapOf(
    "config" to configs.flatMap { listOf(it.name, it.name, it.name) },
    "metric" to configs.flatMap { listOf("Min", "Max", "Avg") },
    "time_ms" to configs.flatMap { listOf(it.minCallTime, it.maxCallTime, it.avgCallTime) }
)

letsPlot(summaryData) { x = "config"; y = "time_ms"; fill = "metric" } +
    geomBar(stat = Stat.identity) +
    ggtitle("Call Time Comparison: Min / Max / Avg") +
    xlab("Configuration") +
    ylab("Time (ms)") +
    theme(axisTextX = elementText(angle = 45, hjust = 1))

#0: Retrofit.singleThread 
 
 
 
 
 
 
 
 
 #1: Retrofit.singleThread 
 
 
 
 
 
 
 
 
 #2: Retrofit.singleThread 
 
 
 
 
 
 
 
 
 #3: Retrofit.dedicatedThreads 
 
 
 
 
 
 
 
 
 #4: Retrofit.dedicatedThreads 
 
 
 
 
 
 
 
 
 #5: Retrofit.dedicatedThreads 
 
 
 
 
 
 
 
 
 #6: Retrofit.dedicatedThreads 
 
 
 
 
 
 
 
 
 #7: Retrofit.dedicatedThreads 
 
 
 
 
 
 
 
 
 #8: Retrofit.dedicatedThreads 
 
 
 
 
 
 
 
 
 #9: Retrofit.dedicatedThreads 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 10,000 
 
 
 
 
 
 
 20,000 
 
 
 
 
 
 
 30,000 
 
 
 
 
 
 
 40,000 
 
 
 
 
 
 
 
 
 Call Time Comparison: Min / Max / Avg 
 
 
 
 
 Time (ms) 
 
 
 
 
 Configuration 
 
 
 
 
 
 
 
 
 metric 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Min 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Max 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Avg

In [50]:
// Box plot: Total time distribution across configurations
val boxPlotData = mapOf(
    "config" to allCallData.map { it.configName },
    "total_time_ms" to allCallData.map { it.totalTimeMs }
)

letsPlot(boxPlotData) { x = "config"; y = "total_time_ms" } +
    geomBoxplot(fill = "#3B9AB2", alpha = 0.7) +
    ggtitle("Total Call Time Distribution") +
    xlab("Configuration") +
    ylab("Total Time (ms)") +
    theme(axisTextX = elementText(angle = 45, hjust = 1))

#0: Retrofit.singleThread 
 
 
 
 
 
 
 
 
 #1: Retrofit.singleThread 
 
 
 
 
 
 
 
 
 #2: Retrofit.singleThread 
 
 
 
 
 
 
 
 
 #3: Retrofit.dedicatedThreads 
 
 
 
 
 
 
 
 
 #4: Retrofit.dedicatedThreads 
 
 
 
 
 
 
 
 
 #5: Retrofit.dedicatedThreads 
 
 
 
 
 
 
 
 
 #6: Retrofit.dedicatedThreads 
 
 
 
 
 
 
 
 
 #7: Retrofit.dedicatedThreads 
 
 
 
 
 
 
 
 
 #8: Retrofit.dedicatedThreads 
 
 
 
 
 
 
 
 
 #9: Retrofit.dedicatedThreads 
 
 
 
 
 
 
 
 
 
 
 5,000 
 
 
 
 
 
 
 10,000 
 
 
 
 
 
 
 15,000 
 
 
 
 
 
 
 20,000 
 
 
 
 
 
 
 
 
 Total Call Time Distribution 
 
 
 
 
 Total Time (ms) 
 
 
 
 
 Configuration

In [51]:
// Timeline: Task completion over time (normalized to start)
val timelineData = allCallData.groupBy { it.configId }.flatMap { (configId, calls) ->
    val minEnqueued = calls.minOf { it.enqueuedAt }
    calls.map { call ->
        mapOf(
            "config" to call.configName,
            "task" to call.callId.substringAfter("#").toInt(),
            "completed_relative_ms" to (call.completedAt - minEnqueued)
        )
    }
}

val flatTimelineData = mapOf(
    "config" to timelineData.map { it["config"] as String },
    "task" to timelineData.map { it["task"] as Int },
    "completed_relative_ms" to timelineData.map { it["completed_relative_ms"] as Long }
)

letsPlot(flatTimelineData) { x = "completed_relative_ms"; y = "task"; color = "config" } +
    geomPoint(size = 3, alpha = 0.6) +
    ggtitle("Task Completion Timeline (normalized to test start)") +
    xlab("Time from start (ms)") +
    ylab("Task Number")

5,000 
 
 
 
 
 
 
 
 
 10,000 
 
 
 
 
 
 
 
 
 15,000 
 
 
 
 
 
 
 
 
 20,000 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 2 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 6 
 
 
 
 
 
 
 8 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 12 
 
 
 
 
 
 
 14 
 
 
 
 
 
 
 16 
 
 
 
 
 
 
 18 
 
 
 
 
 
 
 
 
 Task Completion Timeline (normalized to test start) 
 
 
 
 
 Task Number 
 
 
 
 
 Time from start (ms) 
 
 
 
 
 
 
 
 
 config 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 #0: Retrofit.singleThread 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 #1: Retrofit.singleThread 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 #2: Retrofit.singleThread 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 #3: Retrofit.dedicatedThreads 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 #4: Retrofit.dedicatedThreads 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 #5: Retrofit.dedicatedThreads 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 #6: Retrofit.dedicatedThreads 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 #7: Retrofit.dedicatedThreads 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 #8: Retrofit.dedicatedThreads 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 #9: Retrofit.dedicatedThreads

In [55]:
// Waiting vs Execution time breakdown (averaged per config)
val breakdownData = configs.flatMap { config ->
    val configCalls = allCallData.filter { it.configId == config.id }
    val avgWaitingTime = configCalls.map { it.waitingTimeMs }.average()
    val avgExecTime = configCalls.map { it.execTimeMs }.average()
    
    listOf(
        mapOf("config" to config.name, "type" to "Execution Time", "time_ms" to avgExecTime),
        mapOf("config" to config.name, "type" to "Waiting Time", "time_ms" to avgWaitingTime),
    )
}

val flatBreakdownData = mapOf(
    "config" to breakdownData.map { it["config"] as String },
    "type" to breakdownData.map { it["type"] as String },
    "time_ms" to breakdownData.map { it["time_ms"] as Double }
)

letsPlot(flatBreakdownData) { x = "config"; y = "time_ms"; fill = "type" } +
    geomBar(stat = Stat.identity) +
    ggtitle("Average Time Breakdown: Waiting vs Execution") +
    xlab("Configuration") +
    ylab("Average Time (ms)") +
    theme(axisTextX = elementText(angle = 45, hjust = 1))

#0: Retrofit.singleThread 
 
 
 
 
 
 
 
 
 #1: Retrofit.singleThread 
 
 
 
 
 
 
 
 
 #2: Retrofit.singleThread 
 
 
 
 
 
 
 
 
 #3: Retrofit.dedicatedThreads 
 
 
 
 
 
 
 
 
 #4: Retrofit.dedicatedThreads 
 
 
 
 
 
 
 
 
 #5: Retrofit.dedicatedThreads 
 
 
 
 
 
 
 
 
 #6: Retrofit.dedicatedThreads 
 
 
 
 
 
 
 
 
 #7: Retrofit.dedicatedThreads 
 
 
 
 
 
 
 
 
 #8: Retrofit.dedicatedThreads 
 
 
 
 
 
 
 
 
 #9: Retrofit.dedicatedThreads 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 5,000 
 
 
 
 
 
 
 10,000 
 
 
 
 
 
 
 15,000 
 
 
 
 
 
 
 
 
 Average Time Breakdown: Waiting vs Execution 
 
 
 
 
 Average Time (ms) 
 
 
 
 
 Configuration 
 
 
 
 
 
 
 
 
 type 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Execution Time 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Waiting Time

In [53]:
// Detailed statistics per configuration
println("┌─────┬──────────────────────────────────────────────────────┬──────────┬────────┬────────┬────────┐")
println("│ ID  │ Configuration                                        │ Duration │ Min    │ Max    │ Avg    │")
println("├─────┼──────────────────────────────────────────────────────┼──────────┼────────┼────────┼────────┤")

configs.forEach { config ->
    val configCalls = allCallData.filter { it.configId == config.id }
    val p50 = configCalls.map { it.totalTimeMs }.sorted()[configCalls.size / 2]
    val p95 = configCalls.map { it.totalTimeMs }.sorted()[(configCalls.size * 0.95).toInt()]
    val p99 = configCalls.map { it.totalTimeMs }.sorted()[(configCalls.size * 0.99).toInt()]
    
    println("│ %-3d │ %-52s │ %8d │ %6d │ %6d │ %6d │".format(
        config.id,
        config.name.take(52),
        config.overallDuration,
        config.minCallTime,
        config.maxCallTime,
        config.avgCallTime
    ))
}

println("└─────┴──────────────────────────────────────────────────────┴──────────┴────────┴────────┴────────┘")

┌─────┬──────────────────────────────────────────────────────┬──────────┬────────┬────────┬────────┐
│ ID  │ Configuration                                        │ Duration │ Min    │ Max    │ Avg    │
├─────┼──────────────────────────────────────────────────────┼──────────┼────────┼────────┼────────┤
│ 0   │ #0: Retrofit.singleThread                            │    20111 │   5066 │  20104 │  14087 │
│ 1   │ #1: Retrofit.singleThread                            │    20070 │   1008 │  20068 │  10538 │
│ 2   │ #2: Retrofit.singleThread                            │    20066 │  10035 │  20064 │  17306 │
│ 3   │ #3: Retrofit.dedicatedThreads                        │    20053 │   1004 │  20051 │  10527 │
│ 4   │ #4: Retrofit.dedicatedThreads                        │     4013 │   1005 │   4012 │   2508 │
│ 5   │ #5: Retrofit.dedicatedThreads                        │     2008 │   1004 │   2008 │   1506 │
│ 6   │ #6: Retrofit.dedicatedThreads                        │    20054 │   1005 │  20051 │

In [56]:
// Compare specific configurations (change IDs to compare different configs)
val compareIds = listOf(0, 1) // Edit these to compare different configurations

val compareData = allCallData
    .filter { it.configId in compareIds }

val flatCompareData = mapOf(
    "config" to compareData.map { it.configName },
    "task" to compareData.map { it.callId.substringAfter("#").toInt() },
    "time_ms" to compareData.map { it.totalTimeMs.toDouble() }
)

letsPlot(flatCompareData) { 
    x = "task"; y = "time_ms"; color = "config" 
} +
    geomLine(size = 1.5) +
    geomPoint(size = 2) +
    ggtitle("Task-by-Task Comparison (Config #${compareIds.joinToString(" vs #")})") +
    xlab("Task Number") +
    ylab("Total Time (ms)")

<path d="M14.358883670160058 257.3232090490155 L14.358883670160058 257.3232090490155 L29.473498059802225 241.2483528202003 L44.58811244944439 225.12522374985718 L59.70272683908655 209.00209467951402 L74.81734122872872 192.86287466199488 L89.93195561837088 176.72365464447574 L105.04657000801305 160.5844346269566 L120.1611843976552 144.44521460943747 L135.2757987872974 128.28990364474234 L150.39041317693955 112.1828655215752 L165.5050275665817 96.04364550405606 L180.61964195622386 79.90442548653692 L195.73425634586604 63.781296416193754 L210.8488707355082 47.64207639867465 L225.96348512515036 31.502856381155482 L241.07809951479254 15.363636363636374 L256.1927139044347 15.363636363636374 L271.3073282940769 15.363636363636374 L286.42194268371907 15.363636363636374 L301.5365570733612 15.363636363636374 " fill="none" stroke-width="3.3000000000000003" stroke="rgb(228,26,28)" stroke-opacity="1.0">
 
 
 
 <path d="M14.358883670160058 322.6363636363636 L14.358883670160058 322.6363636363636 L29.473498059802225 306.4810526716685 L44.58811244944439 290.34183265414936 L59.70272683908655 274.2026126366302 L74.81734122872872 258.0473016719351 L89.93195561837088 241.90808165441595 L105.04657000801305 225.7849525840728 L120.1611843976552 209.64573256655368 L135.2757987872974 193.50651254903454 L150.39041317693955 177.3512015843394 L165.5050275665817 161.21198156682027 L180.61964195622386 145.07276154930113 L195.73425634586604 128.933541531782 L210.8488707355082 112.77823056708687 L225.96348512515036 96.63901054956773 L241.07809951479254 80.48369958487262 L256.1927139044347 64.34447956735346 L271.3073282940769 48.22135049701029 L286.42194268371907 32.082130479491184 L301.5365570733612 15.926819514796023 " fill="none" stroke-width="3.3000000000000003" stroke="rgb(55,126,184)" stroke-opacity="1.0">
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 
 
 15 
 
 
 
 
 
 
 
 
 
 
 2,000 
 
 
 
 
 
 
 4,000 
 
 
 
 
 
 
 6,000 
 
 
 
 
 
 
 8,000 
 
 
 
 
 
 
 10,000 
 
 
 
 
 
 
 12,000 
 
 
 
 
 
 
 14,000 
 
 
 
 
 
 
 16,000 
 
 
 
 
 
 
 18,000 
 
 
 
 
 
 
 20,000 
 
 
 
 
 
 
 
 
 Task-by-Task Comparison (Config #0 vs #1) 
 
 
 
 
 Total Time (ms) 
 
 
 
 
 Task Number 
 
 
 
 
 
 
 
 
 config 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 #0: Retrofit.singleThread 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 #1: Retrofit.singleThread